In [16]:
import pandas as pd
import os

In [17]:
fail_list=[]
def ann_to_dict(nct_id): #took out file path arg
    '''
    nct_id: file name before .ann or .txt
    file_path: pathname to where .ann and .txt files are stored
    return: formatted dictionary
    '''
    
    # change directory
    #os.chdir(file_path)
    
    # read in files
    with open(nct_id+'.ann') as f1:
        ann=f1.read()
        f1.close()
    with open(nct_id+'.txt') as f2:
        txt=f2.read()
        f2.close()
        
    # ANN FILE MANIPULATION
    lines=ann.split('\n')
    ents=[]
    for i in range(len(lines)): 
        line=lines[i].split('\t') 
        if 'T' in line[0]:
            try: 
                start=txt.index(line[2])
                end= start+len(line[2])
                label=line[1].split(' ')[0]
                if label in label_list: 
                    add=True
                    for ent in ents: 
                        if (ent[0]<=start<=ent[1]) or (ent[0]<=end<=ent[1]) or (start<=ent[0]<=end) or (start<=ent[1]<=end): #check if it overlaps existing span
                            add=False 
                            if (start-end) > (ent[1]-ent[0]): #replace existing span if this span is bigger
                                ents.remove(ent)
                                ents.append((start, end, label))

                    if add==True: 
                        ents.append((start, end, label))
                else:
                    fail_list.append(label)
            
            except:
                print('oop')
            
    content={'entities': ents, 'text': txt}
    
    return content

In [18]:
import scispacy #
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.load("en_core_sci_sm") #
doc_bin_train = DocBin() # create a DocBin object for train data
doc_bin_test=DocBin() # create a new DocBin object for test data

In [19]:
doc_list=[]
os.chdir('/Users/meldrumapple/Desktop/Capstone/lct_corpus') 
doc_list=os.listdir()
for i in range(len(doc_list)):
    doc_list[i]=doc_list[i][0:-4]
doc_list= list(set([x for x in doc_list if ('NCT') in x])) 

In [20]:
from spacy.util import filter_spans
import random
import re

In [21]:
label_list=['Allergy','Condition','Contraindication','Drug','Encounter','Indication','Immunization','Observation','Procedure','Age','Birth','Death','Ethnicity','Family-member','Language','Life-Stage-And-Gender','Exception','Negation','Acuteness','Assertion','Modifier','Polarity','Risk','Severity','Stability','Eq-Comparison','Eq-Temporal-Period','Eq-Temporal-Recency','Eq-Temporal-Unit','Eq-Unit']

In [22]:

def doc_to_bin(d, bin_obj):
    '''
    d: <str> name of a text/ann doc pair
    bin_obj: <spacy bin object> bin to add doc to
    '''
    doc_dict=ann_to_dict(d)
    text = doc_dict['text']
    labels = doc_dict['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if (span is not None):
                #print(span)
            ents.append(span)
        else:
            None
    filtered_ents=filter_spans(ents)
    doc.ents=filtered_ents
    bin_obj.add(doc)


In [23]:
#split into test and train
l=len(doc_list)
test_idx=random.sample(doc_list, l//6)
train_idx = [x for x in doc_list if x not in test_idx]

#add train docs
print('train')
for d in train_idx:
    try: 
        doc_to_bin(d, doc_bin_train)
    except:
        train_idx.remove(d)
print(train_idx)

#add test docs
print('test')
for d in train_idx:
    try: 
        doc_to_bin(d, doc_bin_test)
    except:
        test_idx.remove(d)
print(test_idx) 
print('Failed labels:')
print(set(fail_list))

train
['NCT03865056', 'NCT03864510', 'NCT03927469', 'NCT03929952', 'NCT03865693', 'NCT03927612', 'NCT03926013', 'NCT03861988', 'NCT03860792', 'NCT03865979', 'NCT03866876', 'NCT03869827', 'NCT03921892', 'NCT03860467', 'NCT03929848', 'NCT03864393', 'NCT03929523', 'NCT03929991', 'NCT03924934', 'NCT03927937', 'NCT03869125', 'NCT03861910', 'NCT03866252', 'NCT03862352', 'NCT03927287', 'NCT03926143', 'NCT03925467', 'NCT03928093', 'NCT03923413', 'NCT03924076', 'NCT03860779', 'NCT03868176', 'NCT03926975', 'NCT03923764', 'NCT03921099', 'NCT03921112', 'NCT03869047', 'NCT03860597', 'NCT03860974', 'NCT03864861', 'NCT03926351', 'NCT03922672', 'NCT03863158', 'NCT03867526', 'NCT03924882', 'NCT03861442', 'NCT03869112', 'NCT03867201', 'NCT03864731', 'NCT03869749', 'NCT03862183', 'NCT03924466', 'NCT03863548', 'NCT03863210', 'NCT03925818', 'NCT03866096', 'NCT03926910', 'NCT03926676', 'NCT03866707', 'NCT03865186', 'NCT03921658', 'NCT03929419', 'NCT03862989', 'NCT03867721', 'NCT03865030', 'NCT03861741', 'NC

In [24]:
# make yourself a new folder somewhere to store model information 
os.chdir('/Users/meldrumapple/Desktop/Capstone/mod_lct') #change your directory to this folder
doc_bin_train.to_disk('training_data.spacy'); # save the docbin objects to this folder
doc_bin_test.to_disk('test_data.spacy');


Next you need to make a new file with the following text (just create a new text file, copy it in). 

Do this in your chia_mod folder or wherever you saved the training data. 

rename the txt file to 'base_config.cfg'

Next in the train and test lines at the top, you need to replace 'null' with your pathname (as a string!) to your training and testing files, so mine looks like: 

Then you need to open a terminal in jupyter lab, and change your directory to where you stored those files and your config files: 

Then you need to run the following line in your terminal for this notebook, so start a jupyter lab terminal. 

Then if that works run this in the same terminal (this step takes a long time): 

If all the terminal stuff works, try running this to test it: 

In [10]:
import spacy 
import os
os.chdir('/Users/meldrumapple/Desktop/Capstone/lct_mod')
nlp_ner = spacy.load("model-best")

doc= nlp_ner(
    '''
    Patients with biopsy-proven metastatic carcinoid tumors or other neuroendocrine tumors (Islet cell, Gastrinomas and VIPomas) with at least one measurable lesion (other than bone) that has either not been previously irradiated or if previously irradiated has demonstrated progression since the radiation therapy 
The patient has no major impairment of renal or hepatic function, as defined by the following laboratory parameters: total bilirubin <1.5 X ULN; AST, ALT<2.5X ULN (<5 X ULN if liver metastases are present) 
Patients on Sandostatin Lar (long acting somatostatin analogue) must be on a stable dose for 30 days prior to study entry and short acting somatostatin analogues must be judged to be on a clinically stable dose by the investigator prior to study entry 
Must have a life expectancy of greater than three (3) months 
Karnofsky Performance Status > 60 
Female patients must have a negative serum pregnancy test at screening. (Not applicable to patients with bilateral oophorectomy and/or hysterectomy or to those patients who are postmenopausal.) 
'''
)

spacy.displacy.render(doc, style="ent", jupyter=True)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/meldrumapple/Desktop/Capstone/lct_mod'

In [ ]:
doc1=nlp_ner('''
             Inclusion Criteria:
  -  Patients of both sexes with a diagnosis of stable CAD as follows: acute myocardial infarction or percutaneous coronary revascularization or coronary artery bypass graft surgery occured at least 12 months before entering the study or chronic stable angina.
Exclusion Criteria:
  -  Mental illness limiting the capacity of self-care or any condition limiting seriously the life expectancy less than 12 months.''')

spacy.displacy.render(doc1, style="ent", jupyter=True)

In [ ]:
doc2=nlp_ner('''
Inclusion Criteria:
  -  BMI 30-40 kg/m2
  -  BMI 18-25 kg/m2; those with BMI up to 28kg/m2 will be included if waist circumference is <96cm.
  -  Males
  -  Aged 18-60yrs
Exclusion Criteria:
  -  Acute illness in the preceding 6 weeks
  -  Taking regular medication
  -  History of deep vein thrombosis or clotting disorders
  -  Hypertension
  -  Diabetes
  -  Any clinically significant findings at screening
  -  History of substance abuse
  -  Demonstrating factors precluding safe MRI
  -  History of gastrointestinal motility disorders (e.g. gastroesophageal reflux disease -irritable bowel syndrome, gastroparesis, sphincter of Oddi dysfunction, etc.)
  -  Previous thoracic or abdominal surgery.
  -  Those who report having ≤3 bowel movements/week or >2/day.)
''')
spacy.displacy.render(doc2, style="ent", jupyter=True)

In [ ]:
doc4=nlp_ner('''
Inclusion Criteria:
  -  Participants will be male and female subjects who suffered total acute Achilles tendon rupture, and which underwent surgical repair. In addition, to participate in this study all volunteers will need to present medical and/or physiotherapeutic release for physical/sports activities practice.
Exclusion Criteria:
  -  Volunteers that did not have Achilles tendon surgical reconstruction, that did not present medical and/or physiotherapeutic release for physical/sports activities, who have participated in strength training program for the plantar flexors in the last 6 months, patients with diabetic diseases, as well as those with difficulty for understanding 
  and/or executing the test and training protocols in the isokinetic dynamometer will be excluded.
''')
spacy.displacy.render(doc4, style="ent", jupyter=True)

In [ ]:
doc5=nlp_ner('''
Inclusion Criteria (ESRD group):
  -  Age ≥ 18 years,
  -  History of biopsy-proven lupus nephritis
  -  All classes of lupus nephritis
  -  Written informed consent
  -  Affiliation to a social security regime
Exclusion Criteria (ESRD group):
  -  Past-history of kidney transplantation
  -  Active infection
  -  Active allergy (such as hay fever)
  -  Pregnant or breastfeeding women
  -  Protected adults (individuals under guardianship by court order)
Inclusion criteria (non ESRD group):
  -  Age ≥ 18 years,
  -  Biopsy proven ACTIVE lupus nephritis
  -  First event of lupus nephritis
  -  No immunosuppressive therapy (including corticosteroids)
  -  Written informed consent
  -  Affiliation to a social security regime
Exclusion criteria (non ESRD group) :
  -  Immunosuppressive treatment
  -  Relapse of lupus nephritis under treatment
  -  Pregnant or breastfeeding women
  -  Protected adults (individuals under guardianship by court order)
''')
spacy.displacy.render(doc5, style="ent", jupyter=True)

In [ ]:
# model-last with this run has 40% accuracy on the test set